<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Regridding TERRACLIMATE with xesmf

_by Joe Hamman (CarbonPlan), June 29, 2020_

This notebook converts the raw TERAACLIMATE dataset to Zarr format.

**Inputs:**


**Outputs:**

- Cloud copy of TERRACLIMATE

**Notes:**

- No reprojection or processing of the data is done in this notebook.


In [ ]:
pip install -U xarray==0.16.0 --no-deps

In [1]:
import fsspec
import xarray as xr
import xesmf as xe
import numpy as np

from dask.diagnostics import ProgressBar

In [2]:
variables = {
#     'conservative': [
#         "aet",
#         "def",
#         "pet",
#         "ppt",
#         "q",
#         "srad",

#     ],
    'bilinear': [
        "tmax",
        "tmin",
        "pdsi",
        "vap",
        "vpd",
        "ws",
        "soil",
        "swe",
        # move to conservative after scrable is fixed
        "aet",
        "def",
        "pet",
        "ppt",
        "q",
        "srad",
        "awc",
        "elevation"
    ]    
}

In [3]:
# options
name = "terraclimate"
raw_location = f"gs://carbonplan-data/raw/terraclimate/4000m/raster.zarr"
target_grid = 'gs://carbonplan-data/processed/grids/conus/4000m/domain.zarr'
# getting weird errors when writing to carbonplan-data
target_location = f'gs://carbonplan-data/processed/{name}/conus/4000m/raster.zarr'

In [4]:
mapper = fsspec.get_mapper(target_grid)
target_ds = xr.open_zarr(mapper, consolidated=True)#.rename({'xc': 'lon', 'yc': 'lat'})
target_ds

<xarray.Dataset>
Dimensions:  (x: 1209, x_b: 1210, y: 783, y_b: 784)
Coordinates:
  * x        (x) float64 -2.491e+06 -2.487e+06 ... 2.337e+06 2.341e+06
  * y        (y) float64 3.308e+06 3.304e+06 3.3e+06 ... 1.84e+05 1.8e+05
Dimensions without coordinates: x_b, y_b
Data variables:
    area     (y, x) float64 dask.array<chunksize=(196, 303), meta=np.ndarray>
    crs      int64 ...
    lat      (y, x) float64 dask.array<chunksize=(196, 303), meta=np.ndarray>
    lat_b    (y_b, x_b) float64 dask.array<chunksize=(196, 303), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(196, 303), meta=np.ndarray>
    lon_b    (y_b, x_b) float64 dask.array<chunksize=(196, 303), meta=np.ndarray>
    mask     (y, x) int64 dask.array<chunksize=(196, 303), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    history:      created by jhamman, 2020-09-23 19:08:35.897424
    title:        Albers 4k CONUS Domain

In [5]:
mapper = fsspec.get_mapper(raw_location)
ds = xr.open_zarr(mapper, consolidated=True)
ds

<xarray.Dataset>
Dimensions:                 (crs: 1, lat: 4320, lon: 8640, time: 744)
Coordinates:
  * crs                     (crs) int16 3
  * lat                     (lat) float64 89.98 89.94 89.9 ... -89.94 -89.98
  * lon                     (lon) float64 -180.0 -179.9 -179.9 ... 179.9 180.0
  * time                    (time) datetime64[ns] 1958-01-01 ... 2019-12-01
Data variables:
    aet                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    awc                     (lat, lon) float32 dask.array<chunksize=(1440, 1440), meta=np.ndarray>
    def                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    elevation               (lat, lon) float32 dask.array<chunksize=(1440, 1440), meta=np.ndarray>
    pdsi                    (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    pet                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    ppt                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    ppt_station_influence   (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    q                       (time, lat, lon) float64 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    soil                    (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    srad                    (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    swe                     (time, lat, lon) float64 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    tmax                    (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    tmax_station_influence  (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    tmin                    (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    tmin_station_influence  (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    vap                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    vap_station_influence   (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    vpd                     (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>
    ws                      (time, lat, lon) float32 dask.array<chunksize=(12, 1440, 1440), meta=np.ndarray>

In [ ]:
step = 360 / 8640 + 1e-9
global_grid = xe.util.grid_global(step, step)
global_grid = global_grid.isel(y=slice(None, None, -1)).isel(y_b=slice(None, None, -1))
global_grid['lat_b'].values = np.clip(global_grid['lat_b'].values, -90, 90)
display(global_grid)

# check that this grid is a drop in replacement for the source grid
assert np.abs(global_grid.lat.isel(x=0).values - ds.lat.values).max() < 1e-5
assert np.abs(global_grid.lon.isel(y=0).values - ds.lon.values).max() < 1e-5
assert np.abs(global_grid.lat).max().item() <= 90
assert np.abs(global_grid.lat_b).max().item() <= 90

# rename grid variables
source_ds = ds.rename({'lon': 'x', 'lat': 'y'}).assign_coords(coords=global_grid.coords)

In [ ]:
regridders = {}

for method in variables:
    regridders[method] = xe.Regridder(source_ds, target_ds, method, reuse_weights=True)

In [ ]:
temp = []
for method, var_list in variables.items():
    regridder = regridders[method]
    temp.append(regridder(ds[var_list].chunk({'lat': -1, 'lon': -1})))

ds_out = xr.merge(temp, compat='override')

In [ ]:
ds_out

In [ ]:
# fs = fsspec.get_filesystem_class('gs')()
# fs.rm(target_location, recursive=True)

In [ ]:
import dask
from multiprocessing.pool import ThreadPool

with dask.config.set(scheduler='threads', pool=ThreadPool(3)):
    with ProgressBar():
        mapper2 = fsspec.get_mapper(target_location)
        ds_out.to_zarr(mapper2, mode='w', consolidated=True)

In [6]:
mapper2 = fsspec.get_mapper(target_location)
import zarr
zarr.consolidate_metadata(mapper2)

<zarr.hierarchy.Group '/'>